In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from google.colab import userdata
from transformers import DataCollatorForLanguageModeling

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("huypn16/annotated-spoc")
#stability ai crashed, now switching to salesforce codegen
# model_checkpoint = "stabilityai/stable-code-3b"
model_checkpoint = "Salesforce/codegen-350M-mono"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14548 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1778 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['Unnamed: 0', 'probid', 'subid', 'text', 'code', 'input_specs'],
    num_rows: 14548
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, token=userdata.get("huggingface_token"))

tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
max_input_length = 2000
max_output_length = 5000

input_param = "text"
output_param = "code"

#add padding token because of the padding error
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
def preprocess_function(examples):
  pseudo_code = [ex for ex in examples["text"]]
  code = [ex for ex in examples["code"]]
  model_inputs = tokenizer(pseudo_code, padding="max_length", truncation=True, return_tensors="pt")
  with tokenizer.as_target_tokenizer():
    model_outputs = tokenizer(code, padding="max_length", truncation=True, return_tensors="pt")
  model_inputs["labels"] = model_outputs["input_ids"]
  return model_inputs

In [ ]:
preprocess_function(dataset["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': tensor([[17953,   257,  3975,  ..., 50295, 50295, 50295],
        [ 1268,    37,   796,  ..., 50295, 50295, 50295]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 8899,    27,  8841,  ..., 50295, 50295, 50295],
        [ 9979,   493, 45594,  ..., 50295, 50295, 50295]])}

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['Unnamed: 0', "probid", "subid", "text", "code", "input_specs"])

Map:   0%|          | 0/14548 [00:00<?, ? examples/s]

Map:   0%|          | 0/1778 [00:00<?, ? examples/s]

In [ ]:

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
batch_size = 16
learning_rate = 1e-3
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, return_tensors="pt", mlm=False)

In [ ]:
# generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt", pad_to_multiple_of=128)

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.set_per_process_memory_fraction(0.9)
model.to(device)

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./pseudocode_to_code_model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=small_train_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 1.39 GiB is free. Process 5151 has 13.35 GiB memory in use. Of the allocated memory 13.05 GiB is allocated by PyTorch, and 173.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=2)
test_dataloader = DataLoader(small_test_dataset, batch_size=2)

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)


In [ ]:
from transformers import get_scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader)*num_train_epochs
)

In [ ]:
# import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(torch.device("cpu"))

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [ ]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(len(train_dataloader)))
# model.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

In [ ]:
# print(type(train_dataloader))
# print(next(iter(train_dataloader)))

In [ ]:
# batch = next(iter(train_dataloader))
# for i, element in enumerate(batch):
#   if type(element) == type(None):
#     print(i)
#     break

In [ ]:
# from torch.utils.data import TensorDataset
# dataset = TensorDataset(small_train_dataset["input_ids"], small_train_dataset["labels"])
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
# for epoch in range(num_train_epochs):
#   for batch in train_dataloader:
#     # if(type(batch) == type(None)):
#     #   print(batch)
#     #   break
#     # print(type(batch))
#     batch = {k: v.to(device) for k, v in batch.items()}
#     outputs = model(**batch)
#     loss = outputs.loss
#     loss.backward()

#     optimizer.step()
#     lr_scheduler.step()
#     optimizer.zero_grad()
#     progress_bar.update(1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)